Baseline_TF
https://www.kaggle.com/code/ivaneleskin/baseline-tf

In [5]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tqdm import tqdm
import json
import gc
from sklearn.model_selection import train_test_split
# import missingno as msno
import multiprocessing as mp

from pathlib import Path
import datetime

In [7]:
DATA_ROOT = Path('data')
DF_TRAIN = DATA_ROOT / 'train.csv'
train = pd.read_csv(DF_TRAIN)

print("train.shape = {} rows, {} cols".format(*train.shape))
train.info()
train.head()


train.shape = 94477 rows, 4 cols
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94477 entries, 0 to 94476
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   path            94477 non-null  object
 1   participant_id  94477 non-null  int64 
 2   sequence_id     94477 non-null  int64 
 3   sign            94477 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.9+ MB


,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie


In [8]:
class CFG:
    data_path = "data/"
    quick_experiment = False
    is_training = True
    use_aggregation_dataset = True
    num_classes = 250
    rows_per_frame = 543 

def load_relevant_data_subset_with_imputation(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    data.replace(np.nan, 0, inplace=True)
    n_frames = int(len(data) / CFG.rows_per_frame)
    data = data.values.reshape(n_frames, CFG.rows_per_frame, len(data_columns))
    return data.astype(np.float32)

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / CFG.rows_per_frame)
    data = data.values.reshape(n_frames, CFG.rows_per_frame, len(data_columns))
    return data.astype(np.float32)

def read_dict(file_path):
    path = os.path.expanduser(file_path)
    with open(path, "r") as f:
        dic = json.load(f)
    return dic

In [9]:
train = pd.read_csv(f"{CFG.data_path}train.csv")
label_index = read_dict(f"{CFG.data_path}sign_to_prediction_index_map.json")
index_label = {label_index[key]: key for key in label_index}
train["label"] = train["sign"].map(lambda sign: label_index[sign])

features_filename = 'feature_data_keyframes.npy'
labels_filename = 'feature_labels_keyframes.npy'

In [60]:
# %%time
import multiprocessing as mp

def first_and_2_disparate_key_frame(data):
    # Initialize variables to store the maximum distance and the corresponding points
    max_distance = 0
    frame0 = 0
    frame1 = 0
    frame2 = 0
    point1 = None
    point2 = None

    where_are_NaNs = np.isnan(data)
    data[where_are_NaNs] = 0

    # Loop over all pairs of points in the array
    # and Compute the Euclidean distance between the current pair of points
    for i in range(1,data.shape[0]):
        for j in range(i+1, data.shape[0]):
            # Check if both points have all zeros for their coordinates
            if np.all(data[i] == 0) or np.all(data[j] == 0):
                print("Skipping zero-frame")
                continue

            distance = np.linalg.norm(data[j] - data[i])
            # If the distance is larger than the current maximum, update the maximum and the corresponding points
            if distance > max_distance:
                max_distance = distance
                point1 = data[i]
                point2 = data[j]
                frame1 = i
                frame2 = j

    data_new = np.zeros((3,543,3))
    data_new[0] = data[frame0,:,:]
    data_new[1] = data[frame1,:,:]
    data_new[2] = data[frame2,:,:]
    return data_new

def convert_row(row):
    _, row = row
    x = load_relevant_data_subset(f"{CFG.data_path}{row.path}")
    x = first_and_2_disparate_key_frame(x)

    # Feature Converter code - START    
    FRAMES = 3
    # Drop Z coordinate so we multiply by 2
    face_x = x[:,:468,:2].reshape(-1, FRAMES*468*2) # Flatten array 
    lefth_x = x[:,468:489,:2].reshape(-1, FRAMES*21*2)
    pose_x = x[:,489:522,:2].reshape(-1, FRAMES*33*2)
    righth_x = x[:,522:,:2].reshape(-1, FRAMES*21*2)

    # Concatenate features
    xs = [] 
    xs.append(face_x)
    xs.append(lefth_x)
    xs.append(pose_x)
    xs.append(righth_x)
    xfeat = np.zeros(sum(x.shape[1] for x in xs))
    start_index = 0
    for x in xs:
        xfeat[start_index:start_index+x.shape[1]] = x
        start_index += x.shape[1]
    # Feature Converter code - END
    return xfeat, row.label

def convert_and_save_data():
    df = train
    df = pd.read_csv(f"{CFG.data_path}train.csv")
    
    label_index = read_dict(f"{CFG.data_path}sign_to_prediction_index_map.json")
    index_label = {label_index[key]: key for key in label_index}
    df["label"] = df["sign"].map(lambda sign: label_index[sign])    
    
    npdata = np.zeros((df.shape[0], 3258))
    nplabels = np.zeros(df.shape[0])

    # for row in tqdm(df.iterrows(), total = df.shape[0]):
    #     i = row[0]
    #     x, y = convert_row(row)
    #     npdata[i,:] = x
    #     nplabels[i] = y
    
    with mp.Pool(processes=16) as pool:
        results = pool.imap_unordered(convert_row, df.iterrows(), chunksize=250)
        for i, (x,y) in tqdm(enumerate(results), total=df.shape[0]):
            npdata[i,:] = x
            nplabels[i] = y
            
    np.save(features_filename, npdata)
    np.save(labels_filename, nplabels)
convert_and_save_data()

  0%|                                                                                                                                                               | 0/94477 [00:00<?, ?it/s]

(3, 33, 2)
(3258,)


In [ ]:
X = np.load(features_filename)
y = np.load(labels_filename)

## Save number of frames of each training sample for data analysis
# train["num_frames"] = num_frames
# print(train["num_frames"].describe())
# train.to_csv("train.csv", index=False)

print(X.shape, y.shape)

In [ ]:
from keras.utils import plot_model

# model = tf.keras.Sequential([
#     tf.keras.layers.GRU(256, return_sequences=True, input_shape=[None, 3]),
#     tf.keras.layers.GRU(256),
#     tf.keras.layers.Dense(256, activation="relu"),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(128, activation="relu"),
#     tf.keras.layers.Dense(128, activation="relu"),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(64, activation="relu"),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(64, activation="relu"),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(250, activation="softmax"),
# ])

model = tf.keras.Sequential([    
    tf.keras.layers.Dense(512),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("gelu"),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(256),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("gelu"),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(250, activation="softmax"),
])


model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
    optimizer='adam',
    metrics=[
        "accuracy",
        # tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name="top-05-acc"),
        # tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10, name="top-10-acc"),
    ]
)

# model.summary()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=27, stratify=y)

# Double the training data by mirroring the coordinates over the x-axis
# X_train_mirrored = X_train.copy()
# X_train_mirrored[:,:,0] *= -1 # flip over x-axis to mirror the data
# y_train = np.concatenate((y_train, y_train))
# X_train = np.concatenate((X_train, X_train_mirrored))

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("model.h5", restore_best_weights=True),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
    ),
    tf.keras.callbacks.TensorBoard(log_dir='logs/fit/' + datetime.datetime.now().strftime("%Y$Ym%d-%H%M%S-keyframes"), histogram_freq=1),
]
history = model.fit(X_train, y_train, epochs=200, validation_data=(X_val, y_val), batch_size=64, callbacks=callbacks)    


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
print(history.history.keys())

In [ ]:
# summarize history for accuracy
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## def get_model():
    return model

In [ ]:
get_model().summary()

In [17]:
# inputs = tf.keras.Input((543, 3), dtype=tf.float32, name="inputs")
# x = tf.where(tf.math.is_nan(inputs), tf.zeros_like(inputs), inputs)
# x = tf.reduce_mean(x, axis=0, keepdims=True)

# # for i in range(1, len(model.layers)):
# x = model.layers[0](x)
# x = model.layers[1](x)
# x = model.layers[2](x)

In [29]:
def get_inference_model(model):
    inputs = tf.keras.Input((543, 3), dtype=tf.float32, name="inputs")
    x = tf.where(tf.math.is_nan(inputs), tf.zeros_like(inputs), inputs)
    x = tf.reduce_mean(x, axis=0, keepdims=True)
    
    for i in range(0, len(model.layers)):
        x = model.layers[i](x)
    output = tf.keras.layers.Activation(activation="linear", name="outputs")(x)
    inference_model = tf.keras.Model(inputs=inputs, outputs=output) 
    inference_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
    
    return inference_model


In [19]:
inference_model = get_inference_model(model)
inference_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, 543, 3)]     0           []                               
                                                                                                  
 tf.math.is_nan_1 (TFOpLambda)  (None, 543, 3)       0           ['inputs[0][0]']                 
                                                                                                  
 tf.zeros_like_1 (TFOpLambda)   (None, 543, 3)       0           ['inputs[0][0]']                 
                                                                                                  
 tf.where_1 (TFOpLambda)        (None, 543, 3)       0           ['tf.math.is_nan_1[0][0]',       
                                                                  'tf.zeros_like_1[0][0]',    

In [30]:
converter = tf.lite.TFLiteConverter.from_keras_model(inference_model)
tflite_model = converter.convert()
model_path = "model.tflite"
# Save the model.
with open(model_path, 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp1ibqbx5a/assets


INFO:tensorflow:Assets written to: /tmp/tmp1ibqbx5a/assets
2023-03-04 20:00:43.060194: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-04 20:00:43.060240: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-04 20:00:43.060400: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp1ibqbx5a
2023-03-04 20:00:43.071015: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-04 20:00:43.071046: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp1ibqbx5a
2023-03-04 20:00:43.111780: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-03-04 20:00:43.191194: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp1ibqbx5a
2023-03-04 20:00:43.253644: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

/bin/bash: /home/victor/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
total 643M
-rw-rw-r-- 1 victor victor 351K Mar  4 16:56 animations.ipynb
-rw-rw-r-- 1 victor victor  62K Mar  4 16:56 baseline-deep_wide.ipynb
-rw-rw-r-- 1 victor victor 262K Mar  4 16:56 baseline-GRU-avg_interpolated_frames.ipynb
-rw-rw-r-- 1 victor victor 236K Mar  4 18:57 baseline-GRU-avg_interpolated_frames-split_inputs.ipynb
-rw-rw-r-- 1 victor victor 147K Mar  4 20:00 baseline-GRU-Beefier.ipynb
-rw-rw-r-- 1 victor victor 140K Mar  4 16:56 baseline-GRU.ipynb
-rw-rw-r-- 1 victor victor 8.9K Mar  4 16:56 environment.yml
-rw-rw-r-- 1 victor victor  30M Mar  4 18:20 model-Copy1.h5
-rw-rw-r-- 1 victor victor 8.6M Mar  4 19:53 model.h5
-rw-rw-r-- 1 victor victor 3.4M Mar  4 20:00 model.tflite
-rw-rw-r-- 1 victor victor 8.6M Mar  4 19:45 model-v1.h5
-rw-rw-r-- 1 victor victor 3.4M Mar  4 20:01 submission.zip
-rw-rw-r-- 1 victor victor  17K Mar  4 16:56 transformer-based-

In [33]:
import tflite_runtime.interpreter as tflite
interpreter = tflite.Interpreter(model_path)
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")
corr = 0
wrong = []
for i in tqdm(range(100)):
    frames = load_relevant_data_subset(f"{CFG.data_path}{train.iloc[i].path}")
    output = prediction_fn(inputs=frames)
    sign = np.argmax(output["outputs"])
    if index_label[sign] == train.iloc[i].sign:
        corr += 1
    else:
        wrong.append((index_label[sign], train.iloc[i].sign, train.iloc[i].path))
    print(f"Predicted label: {index_label[sign]}, Actual Label: {train.iloc[i].sign}")
    total = i

  3%|███▍                                                                                                             | 3/100 [00:00<00:07, 13.30it/s]

Predicted label: brown, Actual Label: blow
Predicted label: shirt, Actual Label: wait
Predicted label: rain, Actual Label: cloud
Predicted label: bird, Actual Label: bird


  7%|███████▉                                                                                                         | 7/100 [00:00<00:06, 15.01it/s]

Predicted label: owie, Actual Label: owie
Predicted label: duck, Actual Label: duck
Predicted label: please, Actual Label: minemy
Predicted label: red, Actual Label: lips


 11%|████████████▎                                                                                                   | 11/100 [00:00<00:05, 15.41it/s]

...

Predicted label: ride, Actual Label: jump
Predicted label: sleepy, Actual Label: sleep
Predicted label: pretty, Actual Label: sun
Predicted label: first, Actual Label: first


 95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▍     | 95/100 [00:06<00:00, 15.34it/s]

Predicted label: callonphone, Actual Label: yellow
Predicted label: brown, Actual Label: brother
Predicted label: grass, Actual Label: grass
Predicted label: uncle, Actual Label: uncle


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.48it/s]

Predicted label: fish, Actual Label: fish
Predicted label: lamp, Actual Label: scissors
Predicted label: cowboy, Actual Label: cowboy
Predicted label: finish, Actual Label: snow


In [34]:
r = f'''
{total=}
{corr=}\t Percent: {corr/total:%}
'''
print(r)


total=99
corr=54	 Percent: 54.545455%



In [35]:
wrongs = pd.DataFrame(data=wrong, columns=['pred','true','path'])
wrongs.groupby('pred').count().reset_index().sort_values('true')

,pred,true,path
0,TV,1,1
20,owl,1,1
21,pajamas,1,1
22,please,1,1
23,pretty,1,1
24,puzzle,1,1
25,rain,1,1
19,outside,1,1
26,read,1,1
28,ride,1,1


In [31]:
import zipfile
zipfile.ZipFile('submission.zip', mode='w').write('model.tflite')

In [ ]:
!ls -lh